# M.Lytova, M.Spanner, I.Tamblyn. *Deep learning and high harmonic generation* (2020)
## Codes for Section IV.B : *Estimating $\lambda$-set values*


##Headers and constants

In [ ]:
from google.colab import files
import numpy as np
import tensorflow as tf
from keras.layers import Input, Dense, Dropout, Conv1D, MaxPooling1D, UpSampling1D, Flatten, Reshape
from keras.models import Model
from keras.optimizers import Nadam, Adam
from tensorflow.keras import initializers
from keras import objectives
from keras.losses import mean_squared_error
from keras.callbacks import TensorBoard
from keras import backend as K
import argparse
import matplotlib.pyplot as plt
import time

In [ ]:
PI = 3.14159265359

t_n_points = 4096   # number of nodes in time
t_n = np.linspace(0, 800, t_n_points)/41.341    # grid in time, Tmax = 800 a.u. = 19.35 fs

n_train = 30000   # training set size
n_test = 1000     # testing set size

##Loading a training set

In [ ]:
param_train = np.zeros((n_train, 3))
path2param = f"/hhg_reduced/param.dat"
param_train = np.loadtxt(path2param, delimiter = ",", max_rows = n_train) 

In [ ]:
y_train = np.zeros((n_train, t_n_points))
path2load0 = f"/hhg_reduced/hhg"

tic = time.perf_counter()

for i in range(n_train): 
    path2load = path2load0 + str(i+1) + '.dat'    
    load_data = np.loadtxt(path2load)
    y_train[i] = load_data[0:t_n_points] * np.sin(PI*t_n/Tmax) 
    if (round(i/1000)==i/1000):
        print(i)        

toc = time.perf_counter()
print(f"Training set preparation time {toc - tic:0.4f} seconds")   

In [ ]:
def plot_train_example(i):
    plt.figure(figsize=(16,5), constrained_layout=False)    
    plt.plot(t_n, y_train[i], color='green')
    plt.xlim(0, 20)
    plt.xticks(np.arange(0, 20, 2.0))
    plt.grid()
    plt.show() 
    plt.close()

### Drawing of a randomly chosen $d_k(t)$

In [ ]:
i_show = np.random.randint(0, n_train-1)
print("theta = ", param_train[i_show,0]*180/PI, ", R = ", param_train[i_show,1], "a.u., I = ", (param_train[i_show,2]/5.338027e-2)**2, "e14 W/cm^2")
plot_train_example(i_show)

##Loading a testing set

In [ ]:
param_test = np.zeros((n_test, 3))
path2param = f"/hhg_reduced/param.dat"
param_test = np.loadtxt(path2param, delimiter = ",", skiprows = n_train, max_rows = n_test) 

print("Testing param set size: ", param_test.shape)

Testing param set size:  (1000, 3)


In [ ]:
y_test = np.zeros((n_test, t_n_points))
path2load0 = f"/hhg_reduced/hhg"

for i in range(n_test): 
    path2load = path2load0 + str(i+1+n_train) + '.dat'    
    load_data = np.loadtxt(path2load)
    y_test[i] = load_data[0:t_n_points] * np.sin(PI*t_n/Tmax)    

##Normalizing before training

In [ ]:
max_E0 = np.amax(param_train[:,2])
min_E0 = np.amin(param_train[:,2])
param_train_norm = (param_train-[0, 1.5, min_E0])/[PI/2, 2.5, (max_E0-min_E0)]
param_test_norm = (param_test-[0, 1.5, min_E0])/[PI/2, 2.5, (max_E0-min_E0)]
y_max = 0.3
y_train_norm = y_train/y_max
y_test_norm = y_test/y_max

## Model

In [ ]:
inputs = Input(shape=(t_n_points,))

x = Dense(512, activation='softplus')(inputs)
x = Reshape((512, 1))(x) 
x = Conv1D(8, 4, activation='softplus', padding='same')(x)
x = Conv1D(8, 4, activation='softplus', padding='same')(x) 
x = MaxPooling1D(2, padding='same')(x)
x = Conv1D(8, 4, activation='softplus', padding='same')(x) 
x = Conv1D(8, 4, activation='softplus', padding='same')(x)
x = MaxPooling1D(2, padding='same')(x)
x = Conv1D(8, 4, activation='softplus', padding='same')(x) 
x = Conv1D(8, 4, activation='softplus', padding='same')(x)
x = MaxPooling1D(2, padding='same')(x)
x = Conv1D(8, 4, activation='softplus', padding='same')(x) 
x = Conv1D(1, 4, activation='softplus', padding='same')(x)
x = Flatten()(x)
x = Dense(64, activation='softplus')(x)
x = Dense(16, activation='softplus')(x)
outputs = Dense(3, activation='softplus')(x)

ModelLambda = Model(inputs, outputs)
opt = Adam(lr=0.0005, amsgrad=True)
ModelLambda.compile(optimizer=opt, loss='mean_squared_error') 

print(ModelLambda.summary())

##Training

*   Training set: 30,000
*   Testing set: 1,000

In [ ]:
def plot_losses2():
    plt.figure(figsize=(8,4))
    plt.plot(np.log10(loss_sum),color='blue')
    plt.plot(np.log10(val_loss_sum),color='red')
    plt.ylabel('log(Loss)', fontsize=12)
    plt.xlabel('Epoch', fontsize=12)
    plt.legend(['training', 'validation'], loc='upper right', fontsize=14)
    plt.show()

In [ ]:
tic = time.perf_counter()

for n in range(3, 10):           # training in a cycle with increasing batch size

      batch_size = 2**n     

      history = ModelLambda.fit(y_train_norm, param_train_norm, 
                             epochs=100,
                             batch_size=batch_size,
                             shuffle=True,
                             validation_data=(y_test_norm, param_test_norm))
      
      path = f"/estimate_thetaRI/model_1" 
      ModelLambda.save(path) 

      loss_save = history.history['loss']
      val_loss_save = history.history['val_loss']
      if n > 3:
            loss_sum = np.concatenate((loss_sum, loss_save), axis = 0)
            val_loss_sum = np.concatenate((val_loss_sum, val_loss_save), axis = 0)
      else:
            loss_sum = loss_save
            val_loss_sum = val_loss_save            
      
      plot_losses2()


toc = time.perf_counter()
print(f"Execution time {toc - tic:0.4f} seconds")
    

##Training and validation losses

In [ ]:
plot_losses2()  

##Prediction

In [ ]:
prediction_norm = ModelLambda.predict(y_test_norm)
prediction = prediction_norm * [PI/2, 2.5, (max_E0-min_E0)] + [0, 1.5, min_E0]

##Predicted vs True

In [ ]:
def plot_predict_true2():
    fig = plt.subplots(3,1,figsize=(4,12),constrained_layout=False)
    plt.subplot(311)
    plt.scatter(param_test[:,0]*180/PI, prediction[:,0]*180/PI, color="blue", s = 0.5)
    plt.ylabel('Predicted', fontsize=14)
    plt.xlim(0,90)
    plt.ylim(0,90)
    plt.xticks(np.arange(0, 91, 30))
    plt.yticks(np.arange(0, 91, 30))
    plt.grid()
    plt.subplot(312)
    plt.scatter(param_test[:,1], prediction[:,1], color="red", s = 0.5)
    plt.ylabel('Predicted', fontsize=14)
    plt.xlim(1.5,4)
    plt.ylim(1.5,4)
    plt.xticks(np.arange(1.5, 4.1, 0.5))
    plt.yticks(np.arange(1.5, 4.1, 0.5))
    plt.grid()
    plt.subplot(313)
    (param_test[i,2]/5.338027e-2)**2
    plt.scatter((param_test[:,2]/5.338027e-2)**2, (prediction[:,2]/5.338027e-2)**2, color="green", s = 0.5)
    plt.ylabel('Predicted', fontsize=14)
    plt.xlabel('True', fontsize=14)
    plt.xlim(1,4)
    plt.ylim(1,4)
    plt.xticks(np.arange(1, 4.1, 1))
    plt.yticks(np.arange(1, 4.1, 1))
    plt.grid()  
    plt.show() 
    plt.close()

plot_predict_true2()    